<a href="https://colab.research.google.com/github/noktavirgul/Dil-analizi-ile-alzheimer-risk-tespiti/blob/main/alzheimer_zamir_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zamir Kullanımı ile Alzheimer Riski Tahmini
Bu notebook, Türkçe cümlelerdeki zamir kullanımlarına göre Alzheimer riski tahmini için BERT modeli eğitir.

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install transformers datasets

In [ ]:
pip install --upgrade accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
from google.colab import files
import pandas as pd

# Dosyayı yükle
uploaded = files.upload()

# Yüklenen dosyanın adını al
filename = list(uploaded.keys())[0]

# Excel dosyasını oku
df = pd.read_excel(filename)

# Sütun isimlerine bak
print(df.columns)


Saving zamir_veriseti.xlsx to zamir_veriseti.xlsx
Index(['Metin', 'Etiket'], dtype='object')


In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

df = pd.read_excel("zamir_veriseti.xlsx")
df = df.rename(columns={"Cümle": "text", "Etiket": "label"})
df["label"] = df["label"].astype(int)
df.head()

Saving zamir_veriseti.xlsx to zamir_veriseti (1).xlsx


,Metin,label
0,"Ben onu aldım, sonra o bana onu verdi.",1
1,"Bugün pazara gittim, elma ve domates aldım.",0
2,"O şey vardı ya, onu bulamıyorum. Onu ben koyma...",1
3,"Annemle sinemaya gittik, çok güzeldi.",0
4,Şeyi getirdin mi? Hani dün konuştuğumuz şeyi.,1


In [ ]:
import pandas as pd
from google.colab import files

# Dosya yükleme butonunu getir
uploaded = files.upload()

# Excel dosyasını oku
df = pd.read_excel("zamir_veriseti.xlsx")

# Kolon adlarını uygun hale getir
df = df.rename(columns={"Cümle": "text", "Etiket": "label"})

# Etiketleri int yap
df["label"] = df["label"].astype(int)

df.head()


Saving zamir_veriseti.xlsx to zamir_veriseti (2).xlsx


,Metin,label
0,"Ben onu aldım, sonra o bana onu verdi.",1
1,"Bugün pazara gittim, elma ve domates aldım.",0
2,"O şey vardı ya, onu bulamıyorum. Onu ben koyma...",1
3,"Annemle sinemaya gittik, çok güzeldi.",0
4,Şeyi getirdin mi? Hani dün konuştuğumuz şeyi.,1


In [ ]:
print(df.columns)


Index(['Metin', 'label'], dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Metin"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:
import torch

class AlzheimerDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AlzheimerDataset(train_encodings, train_labels)
test_dataset = AlzheimerDataset(test_encodings, test_labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)


In [ ]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

trainer.train()


Step,Training Loss
10,0.747500
20,0.694600
30,0.675800
40,0.677000
50,0.656300
60,0.670300
70,0.604000
80,0.636600
90,0.616500
100,0.617900


Step,Training Loss
10,0.747500
20,0.694600
30,0.675800
40,0.677000
50,0.656300
60,0.670300
70,0.604000
80,0.636600
90,0.616500
100,0.617900


TrainOutput(global_step=3000, training_loss=0.08595723913335436, metrics={'train_runtime': 7885.0114, 'train_samples_per_second': 3.041, 'train_steps_per_second': 0.38, 'total_flos': 246461058888000.0, 'train_loss': 0.08595723913335436, 'epoch': 20.0})

In [ ]:
from sklearn.metrics import f1_score

# Gerçek etiketler
y_true = [0, 1, 1, 0, 1, 0]  # örnek gerçek etiketler

# Modelin tahminleri
y_pred = [0, 0, 1, 0, 1, 1]  # modelden aldığın tahminler

# F1 skorunu hesapla
f1 = f1_score(y_true, y_pred)

print(f"F1 Skoru: {f1:.4f}")


F1 Skoru: 0.6667


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

model.save_pretrained("model_checkpoint")
tokenizer.save_pretrained("model_checkpoint")


('model_checkpoint/tokenizer_config.json',
 'model_checkpoint/special_tokens_map.json',
 'model_checkpoint/vocab.txt',
 'model_checkpoint/added_tokens.json')

In [ ]:
import shutil

shutil.make_archive("model_checkpoint", 'zip', "model_checkpoint")


'/content/model_checkpoint.zip'

In [ ]:
from google.colab import files

files.download("model_checkpoint.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>